# Data wrangling project - Dog rating data set from Twitter

In [2]:
# import all necessary libraries
import pandas as pd
import requests
import tweepy
import os
import json

# Gather data

## 1. The WeRateDogs Twitter archive from csv file

In [3]:
Dogarchive=pd.read_csv('twitter-archive-enhanced.csv')
Dogarchive.head()
Dogarchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

## 2. Programatic download of tsv data with Dog image urls

In [4]:
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)
folder_name='tweet_image'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
with open(os.path.join(folder_name,url.split('/')[-1]),mode='wb') as file:
    file.write(response.content)

In [5]:
# storing the tsv data into a dataframe
imagepred=pd.read_csv(os.path.join(folder_name,"image-predictions.tsv"),delimiter="\t")

## 2. Data from Twitter

In [6]:
consumer_key = 'qMvi7Z4iRPMGKYgkXrDVJdbL9'
consumer_secret = 'IXhEjTP8NXDkZzdAVjFSaPtRjZQdrtxaABuvMKO6hS4cAX7Jee'
access_token = '100096730-1QZhfMa4QD5060D5QVxiUH1bbUwu3isJJlL16izg'
access_secret = 'BGI9ulFajY8ej8Tm4cZP1yKluUDK62YbidNlBFaGxOmec'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,parser = tweepy.parsers.JSONParser(),wait_on_rate_limit = True,wait_on_rate_limit_notify = True)

In [7]:
tweet_id=Dogarchive['tweet_id']
list_of_tweets=[]
no_tweets_found=[]
for tweet in tweet_id:
    try:
        list_of_tweets.append(api.get_status(tweet))
    except Exception as e:
        no_tweets_found.append(tweet)

Rate limit reached. Sleeping for: 668
Rate limit reached. Sleeping for: 662


In [8]:
print("Number of tweets identified ",len(list_of_tweets))
print("Number of tweets not identified ",len(no_tweets_found))

Number of tweets identified  2329
Number of tweets not identified  27


In [9]:
list_of_tweets[0]

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 540, 'h': 528, 'resize': 'fit'}}}]},


In [10]:
# store json data into text file
with open('tweet_json.txt',"w") as file:
    for tweet_data in list_of_tweets:
        data=json.dumps(tweet_data)
        file.write(data+"\n")

In [11]:
# Create a dataframe from the tweet data that we just retrieved.
tweet_data = []
with open('tweet_json.txt') as json_file:
    for json_obj in json_file:
        tweet_shortdata_dict = {}
        tweet_alldata_dict = json.loads(json_obj)

        # Get all of the data we're interested in
        tweet_shortdata_dict['tweet_id'] = tweet_alldata_dict['id']
        tweet_shortdata_dict['retweet_count'] = tweet_alldata_dict['retweet_count']
        tweet_shortdata_dict['favorite_count'] = tweet_alldata_dict['favorite_count']
        whole_tweet = tweet_alldata_dict['text']
        tweet_shortdata_dict["short_url"] = whole_tweet[whole_tweet.find('https'):]
        tweet_shortdata_dict['followers_count'] = tweet_alldata_dict['user']['followers_count']
        tweet_shortdata_dict['friends_count'] = tweet_alldata_dict['user']['friends_count']
        whole_source = tweet_alldata_dict['source']
        tweet_shortdata_dict['only_device'] = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        tweet_shortdata_dict['retweeted_status'] = tweet_alldata_dict.get('retweeted_status', 'Original tweet')
        if tweet_shortdata_dict['retweeted_status']!="Original tweet":
            tweet_shortdata_dict['retweeted_status']="Retweet"
        # Append it to the data gathering list
        tweet_data.append(tweet_shortdata_dict)

# Create a dataframe from the data
tweet_data_df = pd.DataFrame(tweet_data)
tweet_data_df.head()

,tweet_id,retweet_count,favorite_count,short_url,followers_count,friends_count,only_device,retweeted_status
0,892420643555336193,7613,35897,https://t.co/MgUWQ76dJU,8810343,17,Twitter for iPhone,Original tweet
1,892177421306343426,5637,30971,https://t.co/aQFSeaCu9L,8810343,17,Twitter for iPhone,Original tweet
2,891815181378084864,3729,23306,https://t.co/r0YlrsGCgy,8810343,17,Twitter for iPhone,Original tweet
3,891689557279858688,7777,39165,https://t.co/tD36da7qLQ,8810971,17,Twitter for iPhone,Original tweet
4,891327558926688256,8385,37439,https://t.co/0g0KMIVXZ3,8810343,17,Twitter for iPhone,Original tweet


In [12]:
tweet_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2329 non-null   int64 
 1   retweet_count     2329 non-null   int64 
 2   favorite_count    2329 non-null   int64 
 3   short_url         2329 non-null   object
 4   followers_count   2329 non-null   int64 
 5   friends_count     2329 non-null   int64 
 6   only_device       2329 non-null   object
 7   retweeted_status  2329 non-null   object
dtypes: int64(5), object(3)
memory usage: 145.7+ KB


# Assessing data

## Manual assessment

### Each dataframe is displayed in the Jupyter Notebook for visual assessment purposes.

In [13]:
Dogarchive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


In [14]:
imagepred

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [15]:
tweet_data_df

,tweet_id,retweet_count,favorite_count,short_url,followers_count,friends_count,only_device,retweeted_status
0,892420643555336193,7613,35897,https://t.co/MgUWQ76dJU,8810343,17,Twitter for iPhone,Original tweet
1,892177421306343426,5637,30971,https://t.co/aQFSeaCu9L,8810343,17,Twitter for iPhone,Original tweet
2,891815181378084864,3729,23306,https://t.co/r0YlrsGCgy,8810343,17,Twitter for iPhone,Original tweet
3,891689557279858688,7777,39165,https://t.co/tD36da7qLQ,8810971,17,Twitter for iPhone,Original tweet
4,891327558926688256,8385,37439,https://t.co/0g0KMIVXZ3,8810343,17,Twitter for iPhone,Original tweet
...,...,...,...,...,...,...,...,...
2324,666049248165822465,40,96,https://t.co/4B7cOc1EDq,8810362,17,Twitter for iPhone,Original tweet
2325,666044226329800704,130,269,https://t.co/DWnyCjf2mx,8810362,17,Twitter for iPhone,Original tweet
2326,666033412701032449,41,111,https://t.co/y671yMhoiR,8810362,17,Twitter for iPhone,Original tweet
2327,666029285002620928,42,120,https://t.co/r7mOb2m0UI,8810362,17,Twitter for iPhone,Original tweet


## Programmatic assessment¶

### Assess the data using panda functions

In [16]:
Dogarchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [17]:
sum(Dogarchive['tweet_id'].duplicated())

0

In [18]:
numerators=Dogarchive.rating_numerator.unique()
Dogarchive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [19]:
#print whole text in order to verify numerators and denominators
indexes=[]
for j,i in enumerate(numerators):
    indexes.append(Dogarchive[i == Dogarchive['rating_numerator']].index.values[0])
    print(j+1,Dogarchive['rating_numerator'][indexes[j]],Dogarchive['text'][indexes[j]],"\n")

1 13 This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU 

2 12 This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB 

3 14 This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A 

4 5 This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948 

5 17 @roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s 

6 11 Please don't send in photos without dogs in them. We're not @porch_rates. Insubordinate and churlish. Pretty good porch tho 11/10 https://t.co/HauE8M3Bu4 

7 10 @ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs 

8 420 @dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research 


> some of the numerators seems to be fractional. due to averaging of multiple dogs

In [20]:
# fractional ratings
with pd.option_context('max_colwidth', 150):
    display(Dogarchive[Dogarchive['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://...,27,10
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10


In [21]:
denominators=Dogarchive.rating_denominator.unique()

In [22]:
indexes=[]
for j,i in enumerate(denominators):
    indexes.append(Dogarchive[i == Dogarchive['rating_denominator']].index.values[0])
    print(j,Dogarchive['rating_denominator'][indexes[j]],Dogarchive['text'][indexes[j]])

0 10 This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU
1 0 @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
2 15 @docmisterio account started on 11/15/15
3 70 The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd
4 7 Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx
5 11 RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
6 150 Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
7 170 Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
8 20 Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a
9 50 This is Bluebert. He just

In [23]:
imagepred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
100,667782464991965184,https://pbs.twimg.com/media/CURwm3cUkAARcO6.jpg,1,lorikeet,0.466149,False,hummingbird,0.083011,False,African_grey,0.054247,False
949,704847917308362754,https://pbs.twimg.com/media/CcgfcANW4AA9hzr.jpg,1,golden_retriever,0.857240,True,Labrador_retriever,0.135460,True,Tibetan_mastiff,0.001903,True
451,674743008475090944,https://pbs.twimg.com/media/CV0rL7RWEAAbhqm.jpg,1,Bernese_mountain_dog,0.583054,True,Shetland_sheepdog,0.065990,True,Greater_Swiss_Mountain_dog,0.065236,True
1579,796177847564038144,https://pbs.twimg.com/media/Cwx99rpW8AMk_Ie.jpg,1,golden_retriever,0.600276,True,Labrador_retriever,0.140798,True,seat_belt,0.087355,False
1184,738883359779196928,https://pbs.twimg.com/media/CkEKe3QWYAAwoDy.jpg,2,Labrador_retriever,0.691137,True,golden_retriever,0.195558,True,Chesapeake_Bay_retriever,0.019585,True


In [24]:
imagepred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
sum(imagepred.tweet_id.duplicated()),sum(imagepred.jpg_url.duplicated())

(0, 66)

In [26]:
pd.concat(g for _, g in imagepred.groupby("jpg_url") if len(g) > 1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
480,675354435921575936,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1641,807106840509214720,https://pbs.twimg.com/ext_tw_video_thumb/80710...,1,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True
1864,842892208864923648,https://pbs.twimg.com/ext_tw_video_thumb/80710...,1,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True
1691,815966073409433600,https://pbs.twimg.com/ext_tw_video_thumb/81596...,1,Tibetan_mastiff,0.506312,True,Tibetan_terrier,0.295690,True,otterhound,0.036251,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1992,873697596434513921,https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg,1,laptop,0.153718,False,French_bulldog,0.099984,True,printer,0.077130,False
2051,887473957103951883,https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg,2,Pembroke,0.809197,True,Rhodesian_ridgeback,0.054950,True,beagle,0.038915,True
2055,888202515573088257,https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg,2,Pembroke,0.809197,True,Rhodesian_ridgeback,0.054950,True,beagle,0.038915,True
1033,711694788429553666,https://pbs.twimg.com/tweet_video_thumb/CeBym7...,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False


In [27]:
print(imagepred.p1_dog.value_counts())
print(imagepred.p2_dog.value_counts())
print(imagepred.p3_dog.value_counts())

True     1532
False     543
Name: p1_dog, dtype: int64
True     1553
False     522
Name: p2_dog, dtype: int64
True     1499
False     576
Name: p3_dog, dtype: int64


In [28]:
imagepred.img_num.value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [29]:
tweet_data_df.sample(10)

,tweet_id,retweet_count,favorite_count,short_url,followers_count,friends_count,only_device,retweeted_status
2073,670704688707301377,351,735,https://t.co/I1lMUy1FqH,8810361,17,Twitter for iPhone,Original tweet
1711,679530280114372609,2039,4716,https://t.co/yavgkHYPOC,8810351,17,Twitter for iPhone,Original tweet
1791,676593408224403456,2091,4431,https://t.co/APvLqbEhkF,8810363,17,Vine - Make a Scene,Original tweet
757,776113305656188928,4378,11820,https://t.co/FYT9QBEnKG,8810342,17,Twitter for iPhone,Original tweet
328,832088576586297345,2,62,5,8810343,17,Twitter for iPhone,Original tweet
38,884876753390489601,5032,25880,https://t.co/DbNzLkDIxh,8810343,17,Twitter for iPhone,Original tweet
360,828361771580813312,173,2193,n,8810343,17,Twitter Web Client,Original tweet
2286,666739327293083650,60,222,https://t.co/tI3uFLDHBI,8810362,17,Twitter for iPhone,Original tweet
1534,688547210804498433,687,2562,https://t.co/7ycWCUrjmZ,8810350,17,Twitter for iPhone,Original tweet
409,821421320206483457,5612,0,https://t.co…,8810343,17,Twitter for iPhone,Retweet


In [30]:
tweet_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2329 non-null   int64 
 1   retweet_count     2329 non-null   int64 
 2   favorite_count    2329 non-null   int64 
 3   short_url         2329 non-null   object
 4   followers_count   2329 non-null   int64 
 5   friends_count     2329 non-null   int64 
 6   only_device       2329 non-null   object
 7   retweeted_status  2329 non-null   object
dtypes: int64(5), object(3)
memory usage: 145.7+ KB


In [31]:
tweet_data_df.retweeted_status.value_counts()

Original tweet    2166
Retweet            163
Name: retweeted_status, dtype: int64

In [32]:
tweet_data_df.only_device.value_counts()

Twitter for iPhone     2195
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                10
Name: only_device, dtype: int64

## Quality

### Completeness, validity, accuracy, consistency (content issues)

#### twitter_archive
    1. The dataset contains not only original ratings, but also retweet ratings.
    2. Delete columns like doggo, floofer, pupper and puppo that won't be used for analysis
    3. Erroneous datatypes (doggo, floofer, pupper and puppo columns)
    4. Incorrect number rating numerator
        a. Fractional ratings (indexes 45,340,695,763,1689,1712)
        b. Wrong values (
    5. Separate timestamp into day - month - year (3 columns)
    6. Correct numerators with decimals
    7. Correct denominators other than 10. 
       
#### image_prediction
    8. Drop duplicates in the jpg_url column
    9. Create 1 column for image prediction and 1 column for confidence level
    10. Delete columns that won't be used for analysis

#### tweet_json
    11. Keep original tweets only

## Tidiness
    1. Change tweet_id to type int64 in order to merge with the other 2 tables
    2. All tables should be part of one dataset

## Cleaning Data

In [119]:
Dogarchive_clean = Dogarchive.copy()
imagepred_clean = imagepred.copy()
tweet_data_df_clean = tweet_data_df.copy()

 **1. The dataset contains not only original ratings, but also retweet ratings.**

Define : Delete retweets by filtering the null values in retweeted_status_user_id column

In [120]:
#CODE: 
Dogarchive_clean = Dogarchive_clean[pd.isnull(Dogarchive_clean['retweeted_status_user_id'])]

#TEST
print(sum(Dogarchive_clean.retweeted_status_user_id.value_counts()))

0


In [121]:
Dogarchive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        2175 non-null   object 
 13  doggo                       2175 

**2. Delete columns like doggo, floofer, pupper and puppo that won't be used for analysis**

In [122]:
Dogarchive_clean.drop(columns=['source','in_reply_to_status_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','in_reply_to_status_id','in_reply_to_user_id','expanded_urls'],inplace=True)

In [123]:
#Test
list(Dogarchive_clean)

['tweet_id',
 'timestamp',
 'text',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

**3. Erroneous datatypes (doggo, floofer, pupper and puppo columns)**

Define: Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_group column. 

In [124]:
#CODE: Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_group column
Dogarchive_clean = pd.melt(Dogarchive_clean, id_vars=['tweet_id',                                          
                                                                'timestamp',
                                                                'text',
                                                                'rating_numerator',
                                                                'rating_denominator',
                                                                'name'],
                               var_name='dogs', value_name='dogs_group')

#CODE: drop dogs
Dogarchive_clean = Dogarchive_clean.drop('dogs', axis=1)

#CODE: Sort by dogs_stage then drop duplicated based on tweet_id except the last occurrence
Dogarchive_clean = Dogarchive_clean.sort_values('dogs_group').drop_duplicates(subset='tweet_id',
                                                                                        keep='last')

In [125]:
#TEST
Dogarchive_clean['dogs_group'].value_counts()

None       1831
pupper      234
doggo        75
puppo        25
floofer      10
Name: dogs_group, dtype: int64

In [126]:
Dogarchive_clean.head()

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dogs_group
2095,667443425659232256,2015-11-19 20:44:47 +0000,Exotic dog here. Long neck. Weird paws. Obsess...,6,10,None,None
2094,667453023279554560,2015-11-19 21:22:56 +0000,Meet Cupcake. I would do unspeakable things fo...,11,10,Cupcake,None
2093,667455448082227200,2015-11-19 21:32:34 +0000,This is Reese and Twips. Reese protects Twips....,7,10,Reese,None
2092,667470559035432960,2015-11-19 22:32:36 +0000,This is a northern Wahoo named Kohl. He runs t...,11,10,a,None
2091,667491009379606528,2015-11-19 23:53:52 +0000,Two dogs in this one. Both are rare Jujitsu Py...,7,10,None,None


**4. Correct Incorrect number rating numerator**

    a. Fractional ratings

In [127]:
# Find indexes of text with fractional ratings
with pd.option_context('max_colwidth', 200):
    display(Dogarchive_clean[Dogarchive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,tweet_id,text,rating_numerator,rating_denominator
42,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
3685,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
3708,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10
2733,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
4967,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10


In [128]:
#CODE

#First change numerator and denominators type int to float to allow decimals 
Dogarchive_clean[['rating_numerator', 'rating_denominator']] = Dogarchive_clean[['rating_numerator','rating_denominator']].astype(float)

#Update numerators

Dogarchive_clean.loc[Dogarchive_clean.index==42, 'rating_numerator'] = 13.5
Dogarchive_clean.loc[Dogarchive_clean.index==2733, 'rating_numerator'] = 9.75
Dogarchive_clean.loc[Dogarchive_clean.index==4967, 'rating_numerator'] = 11.27
Dogarchive_clean.loc[Dogarchive_clean.index==3685, 'rating_numerator'] = 9.5
Dogarchive_clean.loc[Dogarchive_clean.index==3708, 'rating_numerator'] = 11.26
Dogarchive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 2095 to 7298
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2175 non-null   int64  
 1   timestamp           2175 non-null   object 
 2   text                2175 non-null   object 
 3   rating_numerator    2175 non-null   float64
 4   rating_denominator  2175 non-null   float64
 5   name                2175 non-null   object 
 6   dogs_group          2175 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 135.9+ KB


In [129]:
#Test
with pd.option_context('max_colwidth', 200):
    display(Dogarchive_clean[Dogarchive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,tweet_id,text,rating_numerator,rating_denominator
42,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
3685,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,9.50,10.0
3708,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0
2733,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
4967,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0


**5. Twitter_archive - Separate timestamp into day - month - year (3 columns)**

Define : First convert timestamp to datetime. Then extract year, month and day to new columns. Finally drop timestamp column.

In [135]:
Dogarchive_clean.head()

,tweet_id,text,rating_numerator,rating_denominator,name,dogs_group,year,month,day
2095,667443425659232256,Exotic dog here. Long neck. Weird paws. Obsess...,6.0,10.0,None,None,2015,11,19
2094,667453023279554560,Meet Cupcake. I would do unspeakable things fo...,11.0,10.0,Cupcake,None,2015,11,19
2093,667455448082227200,This is Reese and Twips. Reese protects Twips....,7.0,10.0,Reese,None,2015,11,19
2092,667470559035432960,This is a northern Wahoo named Kohl. He runs t...,11.0,10.0,a,None,2015,11,19
2091,667491009379606528,Two dogs in this one. Both are rare Jujitsu Py...,7.0,10.0,None,None,2015,11,19


In [134]:
#CODE: convert timestamp to datetime
Dogarchive_clean['timestamp'] = pd.to_datetime(Dogarchive_clean['timestamp'])

#extract year, month and day to new columns
Dogarchive_clean['year'] = Dogarchive_clean['timestamp'].dt.year
Dogarchive_clean['month'] = Dogarchive_clean['timestamp'].dt.month
Dogarchive_clean['day'] = Dogarchive_clean['timestamp'].dt.day

#Delete timestamp column
Dogarchive_clean.drop('timestamp', axis=1,inplace=True)

KeyError: 'timestamp'

6. Twitter_archive - Correc denominators

a. Manually
Five tweets with denominator not equal to 10 for special circunstances. Update both numerators and denominators when necessary. Delete other five tweets because they do not have actual ratings.

In [ ]:
#CODE: Update both numerators and denominators
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 740373189193256964), 'rating_numerator'] = 14
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 740373189193256964), 'rating_denominator'] = 10

Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 682962037429899265), 'rating_numerator'] = 10
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 682962037429899265), 'rating_denominator'] = 10

Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 722974582966214656), 'rating_numerator'] = 13
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 722974582966214656), 'rating_denominator'] = 10

Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 13.5
Dogarchive_clean.loc[(Dogarchive_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10

#CODE: Delete five tweets with no actual ratings
Dogarchive_clean = Dogarchive_clean[Dogarchive_clean['tweet_id'] != 832088576586297345]
Dogarchive_clean = Dogarchive_clean[Dogarchive_clean['tweet_id'] != 810984652412424192]
Dogarchive_clean = Dogarchive_clean[Dogarchive_clean['tweet_id'] != 682808988178739200]
Dogarchive_clean = Dogarchive_clean[Dogarchive_clean['tweet_id'] != 835246439529840640]
Dogarchive_clean = Dogarchive_clean[Dogarchive_clean['tweet_id'] != 686035780142297088]

#TEST: Left only the group dogs for programatically clean
with pd.option_context('max_colwidth', 200):
    display(Dogarchive_clean[Dogarchive_clean['rating_denominator'] != 10][['tweet_id',
                                                                                      'text',
                                                                                      'rating_numerator',
                                                                                      'rating_denominator']])